In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
import re
import string
from collections import Counter
import contractions
from wordcloud import WordCloud

import dash
from dash import html, dcc, Input, Output, State
import pandas as pd
import joblib
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import string
import contractions
import dash_table


# Load model and data
model = joblib.load("../artifacts/model2.pkl")
df = pd.read_csv('../data/validation_data.csv')

# Define valid variation options
variation_options = [
    {'label': 'Charcoal Fabric', 'value': 'charcoal_fabric'},
    {'label': 'Walnut Finish', 'value': 'walnut_finish'},
    {'label': 'Heather Gray Fabric', 'value': 'heather_gray_fabric'},
    {'label': 'Sandstone Fabric', 'value': 'sandstone_fabric'},
    {'label': 'Oak Finish', 'value': 'oak_finish'},
    {'label': 'Black', 'value': 'black'},
    {'label': 'White', 'value': 'white'},
    {'label': 'Black Spot', 'value': 'black_spot'},
    {'label': 'White Spot', 'value': 'white_spot'},
    {'label': 'Black Show', 'value': 'black_show'},
    {'label': 'White Show', 'value': 'white_show'},
    {'label': 'Black Plus', 'value': 'black_plus'},
    {'label': 'White Plus', 'value': 'white_plus'},
    {'label': 'Configuration: Fire TV Stick', 'value': 'fire_tv_stick'},
    {'label': 'Black Dot', 'value': 'black_dot'},
    {'label': 'White Dot', 'value': 'white_dot'}
]

# Initialize Dash app
app = dash.Dash(__name__)
server = app.server

# Define app layout
app.layout = html.Div(
    className='app-container',
    style={
        'background-color': '#1d7874',
        'width': '80%',
        'margin': 'auto',
        'padding': '50px'
    },
    children=[
        html.H1('Amazon Alexa Review', style={'text-align': 'center'}),
        html.Div(
            id='content',
            className='content',
            children=[
                html.Div([
                    html.Label('Rating', style={'font-size': '20px'}),
                    dcc.Slider(
                        id='rating-slider',
                        min=1,
                        max=5,
                        step=1,
                        value=3,
                        marks={i: str(i) for i in range(1, 6)},
                    )
                ], className='rating-container'),
                html.Div([
                    html.Label('Date', style={'font-size': '20px'}),
                    dcc.DatePickerSingle(
                        id='date-picker',
                        date=None
                    )
                ], className='date-container'),
                html.Div([
                    html.Label('Variation', style={'font-size': '20px'}),
                    dcc.Dropdown(
                        id='variation-dropdown',
                        options=variation_options,
                        placeholder='Select variation',
                        style={'width': '100%'}
                    )
                ], className='variation-container'),
                html.Div([
                    html.Label('Review', style={'font-size': '20px'}),
                    dcc.Textarea(
                        id='review-text',
                        value='',
                        style={'width': '100%', 'height': 200}
                    )
                ], className='review-container'),
                html.Button('Submit', id='submit-val', n_clicks=0, style={'margin-top': '10px'}),
                html.Div(id='output-div', style={'margin-top': '20px'}),
                html.H3('Amazon Reviews', style={'margin-top': '20px'}),
                dcc.Loading(
                    id='loading-table',
                    type='circle',
                    children=[
                        dash_table.DataTable(
                            id='review-table',
                            columns=[{"name": i, "id": i} for i in df.columns],
                            data=df.to_dict('records'),
                            row_selectable='single',
                            style_table={
                                'height': '400px',
                                'maxHeight': '400px',
                                'overflowY': 'auto',
                                'backgroundColor': '#1f2937',
                            },
                            style_header={
                                'backgroundColor': '#374151',
                                'color': 'white',
                                'position': 'sticky',
                                'top': 0
                            },
                            style_cell={
                                'minWidth': '150px',
                                'maxWidth': '150px',
                                'whiteSpace': 'normal',
                                'textAlign': 'left',
                                'backgroundColor': '#374151',
                                'color': 'white'
                            },
                            fixed_rows={'headers': True},
                            selected_rows=[],
                        )
                    ]
                )
            ]
        )
    ]
)


@app.callback(
    Output('rating-slider', 'value'),
    Output('date-picker', 'date'),
    Output('variation-dropdown', 'value'),
    Output('review-text', 'value'),
    Input('review-table', 'selected_rows'),
    State('review-table', 'data')
)
def update_inputs(selected_rows, data):
    if selected_rows:
        selected_row = data[selected_rows[0]]
        rating = selected_row['rating']
        date = selected_row['date']
        variation = selected_row['variation']
        review = selected_row['verified_reviews']
        return rating, date, variation, review
    else:
        return 3, None, None, ''

# Callback to process review text and predict sentiment
@app.callback(
    Output('output-div', 'children'),
    Input('submit-val', 'n_clicks'),
    State('review-text', 'value')
)
def update_output(n_clicks, review_text):
    if n_clicks and review_text:
        processed_text = process(review_text)
        vectorized_text = vectorizer.transform([processed_text])
        predicted_sentiment = model.predict(vectorized_text)[0]

        if predicted_sentiment == -1:
            sentiment = "Negative Sentiment"
        elif predicted_sentiment == 0:
            sentiment = "Neutral Sentiment"
        else:
            sentiment = "Positive Sentiment"

        return html.Div([
            html.H3("Predicted Sentiment:"),
            html.P(sentiment)
        ])

    return None


def process(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove emojis
    emoji_pattern = re.compile("["
                              u"\U0001F600-\U0001F64F"  
                              u"\U0001F300-\U0001F5FF"  
                              u"\U0001F680-\U0001F6FF"  
                              u"\U0001F1E0-\U0001F1FF"  
                              u"\U00002500-\U00002BEF"  
                              u"\U00002702-\U000027B0"
                              u"\U00002702-\U000027B0"
                              u"\U000024C2-\U0001F251"
                              u"\U0001f926-\U0001f937"
                              u"\U00010000-\U0010ffff"
                              u"\u2640-\u2642"
                              u"\u2600-\u2B55"
                              u"\u200d"
                              u"\u23cf"
                              u"\u23e9"
                              u"\u231a"
                              u"\ufe0f"  
                              u"\u3030"
                              "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)

    text = contractions.fix(text)

    tokens = text.split()
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]

    processed_text = ' '.join(lemmatized_tokens)
    return processed_text


if __name__ == '__main__':
    app.run_server(port=8011, debug=True)


C:\Users\encry\AppData\Local\Temp\ipykernel_20328\1785201605.py:28: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table
